In [1]:
import numpy as np
import json
import warnings
import tensorflow as tf
from tensorflow.keras import models, Model
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import Dense, Conv2D, Flatten, MaxPool2D, BatchNormalization, Dropout, Input, Concatenate, GlobalAvgPool2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import Sequence
import json
from evaluator import *

In [2]:
VARIABLES = ['ActivityCounts', 'Barometer', 'BloodPerfusion',
             'BloodPulseWave', 'EnergyExpenditure', 'GalvanicSkinResponse', 'HR',
             'HRV', 'RESP', 'Steps', 'SkinTemperature', 'ActivityClass']

In [3]:
GRAYSCALE = False # grayscale or rgb

# Import data

In [4]:
# file path to data folder
path = './Output'

In [5]:
# dimensions
N, HEIGHT, WIDTH, CHANNELS = sum([1 for p in os.listdir(path) if (p[:14] == 'feature_vector' and p[:19] != 'feature_vector_stat')]), \
                             *np.load(path + '/feature_vector0.npy').shape
CHANNELS = len(VARIABLES) if GRAYSCALE else CHANNELS # reduce channels for grayscale

print(N, HEIGHT, WIDTH, CHANNELS)

613 370 497 30


Metadata (subjectID etc.)

In [6]:
with open(path + '/metadata.txt') as f:
    metadata = f.read()

metadata = json.loads(metadata.replace('\'', '\"').replace('False', 'false').replace('True', 'true')) # doesn't accept other chars

In [7]:
subjects = [meta['subjectID'] for meta in metadata]

# CNN

Addditional functions

In [8]:
# image-wise transformer
def rgb2gray(rgb):
    """greyscale = 0.2989 * red + 0.5870 * green + 0.1140 * blue"""
    return np.dot(rgb[:, :, :3], [0.2989, 0.5870, 0.1140])

# loss function
def weighted_cross_entropy(weight):
    def weighted_cross_entropy_with_logits(labels, logits):
        loss = tf.nn.weighted_cross_entropy_with_logits(
            labels, logits, weight
        )
        return loss
    return weighted_cross_entropy_with_logits

# weight (imbalanced classes)
def check_imbalance(path_to_labels, indices):
    """Returns indices of positives/negatives"""
    y = np.empty((len(indices), 2), dtype=int)
    for i, index in enumerate(indices):
        y[i, ] = np.load(path_to_labels + f'/labels{index}.npy', allow_pickle=True)

    positives = np.where(y[:, variable] == 1)[0] # TODO: for now just one variable
    negatives = np.where(y[:, variable] == 0)[0] # TODO: for now just one variable

    return np.array(indices)[positives], np.array(indices)[negatives]

def get_weighting_factor(path, train_set_indices):
    positives, negatives = check_imbalance(path, train_set_indices)
    sample_weight = len(negatives) / len(positives) # for weighted cross-entropy
    return sample_weight

Dataloader (dataset with images too large)

In [9]:
class DataGenerator(Sequence):

    def __init__(self, data_path: str, indices_dataset: list, batch_size=32, dim=(HEIGHT, WIDTH), n_channels=CHANNELS, shuffle=True):
        self.data_path = data_path # path to full dataset
        self.dim = dim # image dimension
        self.batch_size = batch_size
        self.indices_dataset = indices_dataset # indices of full dataset (different for train/validation/test set)
        self.n_channels = n_channels
        self.shuffle = shuffle

        self.on_epoch_end() # shuffle data for each epoch

    def on_epoch_end(self):
        """
        Shuffle data for each epoch
        """
        if self.shuffle:
            np.random.shuffle(self.indices_dataset)

    def __data_generation(self, indices):
        """
        Loads and returns datapoints[indices]
        """
        # init
        X = np.empty((self.batch_size, *self.dim, self.n_channels))
        y = np.empty(self.batch_size, dtype=float) # TODO: int for non-logits

        # load individual datapoints
        for i, index in enumerate(indices):
            images = np.load(self.data_path + f'/feature_vector{index}.npy', allow_pickle=True)
            if GRAYSCALE:
                images_gray = np.empty((HEIGHT, WIDTH, self.n_channels))
                for j in range(len(VARIABLES)):
                    image_rgb = images[:, :, (3 * j): (3 * (j + 1))]
                    image_gray = rgb2gray(image_rgb)
                    images_gray[:, :, j] = image_gray
                images = images_gray

            X[i, ] = images
            y[i] = np.load(self.data_path + f'/labels{index}.npy', allow_pickle=True)[variable] # TODO: for now just one variable

        return X, y

    def __len__(self):
        """
        Number of batches per epoch
        """
        return int(np.floor(len(self.indices_dataset) / self.batch_size))

    def __getitem__(self, index):
        """
        Generates batch[index]
        """
        # calculate indices of batch
        indices = self.indices_dataset[index * self.batch_size:(index + 1) * self.batch_size]

        # generate batch
        X, y = self.__data_generation(indices)

        return X, y

Architecture

In [10]:
# TODO: make possible for grayscale
class ConvNet(tf.keras.Model):

    def __init__(self, name='cnn', **kwargs):
        super(ConvNet, self).__init__(name, **kwargs)

        self.in_shape = (HEIGHT, WIDTH, CHANNELS)
        self.in_shape_mobilenet = (HEIGHT, WIDTH, 3)

        # MobileNetV2 embedding
        self.mobilenet = MobileNetV2(input_shape=self.in_shape_mobilenet, weights='imagenet', include_top=False)
        self.mobilenet._name = 'mobilenet'
        self.mobilenet.trainable = False
        self.finetuning = False
        self.out_shape_mobilenet = self.mobilenet.layers[-1].output_shape # for one spectrogram

        # Concatenation
        self.concat = Concatenate(name='concat')

        # Global pooling
        self.pool = GlobalAvgPool2D(name='global_avg_pool')

        # TODO: more sophisticated dense (dropout, regularizer, init., ...)
        # Fully-connected network
        self.flatten = Flatten(name='flatten', input_shape=(self.out_shape_mobilenet * (CHANNELS // 3), ))
        self.dense = Dense(1, name='dense') # keep logits
        self.out_shape = 1

        # build graph
        self.build_graph()

    def build_graph(self):
        self.build(input_shape=(None, *self.in_shape))
        x = Input(shape=self.in_shape)
        Model(inputs=[x], outputs=self.call(x))

    def set_finetuning(self, mode=True):
        self.finetuning = mode
        self.mobilenet.trainable = mode

        for layers in self.mobilenet.layers:
            layers.trainable = False

        # "activate" last conv layer of MobileNet
        self.mobilenet.layers[-3].trainable = mode
        self.mobilenet.layers[-2].trainable = mode

    def call(self, inputs):
        """
        Model predictions (logits)
        :param inputs: all spectrograms of shape (HEIGHT, WIDTH, CHANNELS)
        :return: class prediction (logits)
        """
        # MobileNetV2 embeddings
        x = [self.mobilenet(inputs[..., i:i+3], training=self.finetuning) for i in range(0, CHANNELS, 3)]

        # Concatenation
        x = self.concat(x)

        # Global pooling
        x = self.pool(x)

        # Fully-connected network
        x = self.flatten(x)
        x = self.dense(x)

        return x

Model

In [11]:
class CNN:

    def __init__(self, path, variable, epochs, learning_rate, batch_size):
        self.model = ConvNet()
        self.path = path
        assert variable in (0, 1)
        self.variable = variable
        self.epochs = epochs
        self.learning_rate = learning_rate
        self.batch_size = batch_size
        self.history = None
        self.train_indices = None
        self.test_indices = None

    def fit(self, train_indices):
        self.train_indices = train_indices

        # training set
        train_dataloader = DataGenerator(self.path, train_indices, batch_size=self.batch_size)

        # weights for loss function
        sample_weights = get_weighting_factor(self.path, train_indices)

        # build model
        self.model.compile(optimizer=Adam(learning_rate=self.learning_rate),
                           loss=weighted_cross_entropy(sample_weights))

        # training
        self.history = self.model.fit_generator(generator=train_dataloader,
                                                epochs=self.epochs)

    def reset(self):
        """Resets model weights"""
        self.model = ConvNet()

    def predict(self, test_indices):
        """Predicts actual class labels (not logits/probability values!)"""
        self.test_indices = test_indices

        # TODO: make more efficient
        # test set + predict
        y_pred = np.empty(len(test_indices), dtype=float)

        for i, index in enumerate(test_indices):
            X_i = np.load(path + f'/feature_vector{index}.npy', allow_pickle=True)

            X_i = tf.expand_dims(X_i, axis=0) # add "batch dimension"
            logits_pred_i = self.model.predict(X_i)

            y_pred[i] = logits_pred_i

        y_probs = tf.math.sigmoid(y_pred) # logits to probs
        y_pred = tf.round(y_probs) # probs to labels

        return y_pred

    def summary(self):
        return self.model.summary()

# CV

In [ ]:
with warnings.catch_warnings():
    warnings.filterwarnings('ignore')

    for variable in (0, 1): # phF, MF
        model = CNN(path, variable=0, epochs=10, learning_rate=1e-3, batch_size=16)

        scores_strat_group_k_fold = stratified_group_k_fold(path=path,
                                                            groups=subjects,
                                                            model=model,
                                                            folds=5,
                                                            images=True,
                                                            verbose=True,
                                                            variable=variable)

        # not a valid CV method (train/test mixing due to overlapping images)
        '''scores_strat_k_fold = stratified_k_fold(path=path,
                                                model=model,
                                                folds=5,
                                                images=True,
                                                verbose=True,
                                                variable=variable)'''

        scores_loso = leave_one_subject_out(path=path,
                                            groups=subjects,
                                            model=model,
                                            images=True,
                                            verbose=True,
                                            variable=variable)

Starting stratified group 5-fold for physical fatigue


  0%|          | 0/5 [00:00<?, ?it/s]

Epoch 1/10
31/31 [==============================] - 54s 657ms/step - loss: 0.4965
Epoch 2/10
31/31 [==============================] - 20s 647ms/step - loss: 0.3856
Epoch 3/10
31/31 [==============================] - 20s 647ms/step - loss: 0.3576
Epoch 4/10
31/31 [==============================] - 20s 647ms/step - loss: 0.3236
Epoch 5/10
31/31 [==============================] - 20s 646ms/step - loss: 0.3477
Epoch 6/10
31/31 [==============================] - 20s 645ms/step - loss: 0.4547
Epoch 7/10
31/31 [==============================] - 20s 646ms/step - loss: 0.3263
Epoch 8/10
31/31 [==============================] - 20s 647ms/step - loss: 0.2906
Epoch 9/10
31/31 [==============================] - 20s 647ms/step - loss: 0.2592
Epoch 10/10
31/31 [==============================] - 20s 645ms/step - loss: 0.2524


 Fold 1 F1: 0.8956603773584907:  20%|██        | 1/5 [04:31<18:06, 271.56s/it]

Epoch 1/10
20/20 [==============================] - 36s 639ms/step - loss: 0.5994
Epoch 2/10
20/20 [==============================] - 13s 643ms/step - loss: 0.4429
Epoch 3/10
20/20 [==============================] - 13s 638ms/step - loss: 0.3469
Epoch 4/10
20/20 [==============================] - 13s 644ms/step - loss: 0.3033
Epoch 5/10
20/20 [==============================] - 13s 640ms/step - loss: 0.2902
Epoch 6/10
20/20 [==============================] - 13s 641ms/step - loss: 0.2813
Epoch 7/10
20/20 [==============================] - 13s 640ms/step - loss: 0.2577
Epoch 8/10
20/20 [==============================] - 13s 642ms/step - loss: 0.2460
Epoch 9/10
20/20 [==============================] - 13s 640ms/step - loss: 0.2387
Epoch 10/10
20/20 [==============================] - 13s 643ms/step - loss: 0.2487


...

Epoch 1/10
32/32 [==============================] - 48s 667ms/step - loss: 0.3860
Epoch 2/10
32/32 [==============================] - 23s 696ms/step - loss: 0.3257
Epoch 3/10
32/32 [==============================] - 22s 669ms/step - loss: 0.2482
Epoch 4/10
32/32 [==============================] - 21s 651ms/step - loss: 0.2365
Epoch 5/10
32/32 [==============================] - 22s 663ms/step - loss: 0.2062
Epoch 6/10
32/32 [==============================] - 21s 651ms/step - loss: 0.2539
Epoch 7/10
32/32 [==============================] - 22s 684ms/step - loss: 0.1925
Epoch 8/10
32/32 [==============================] - 22s 681ms/step - loss: 0.1648
Epoch 9/10
32/32 [==============================] - 22s 660ms/step - loss: 0.1490
Epoch 10/10
32/32 [==============================] - 22s 662ms/step - loss: 0.2056


 Fold 23 F1: 0.3903225806451613:  85%|████████▌ | 23/27 [1:52:07<19:19, 289.99s/it]

Epoch 1/10
29/29 [==============================] - 45s 649ms/step - loss: 0.5189
Epoch 2/10
29/29 [==============================] - 19s 653ms/step - loss: 0.4480
Epoch 3/10
29/29 [==============================] - 19s 650ms/step - loss: 0.4240
Epoch 4/10
29/29 [==============================] - 20s 659ms/step - loss: 0.3837
Epoch 5/10
29/29 [==============================] - 20s 660ms/step - loss: 0.3281
Epoch 6/10
29/29 [==============================] - 20s 659ms/step - loss: 0.3062
Epoch 7/10
29/29 [==============================] - 20s 665ms/step - loss: 0.3153
Epoch 8/10
29/29 [==============================] - 19s 658ms/step - loss: 0.2688
Epoch 9/10
29/29 [==============================] - 20s 672ms/step - loss: 0.2789
Epoch 10/10
29/29 [==============================] - 20s 667ms/step - loss: 0.2451


 Fold 24 F1: 0.7811120032115615:  89%|████████▉ | 24/27 [1:56:31<14:06, 282.33s/it]

Epoch 1/10
36/36 [==============================] - 49s 641ms/step - loss: 0.4149
Epoch 2/10
36/36 [==============================] - 24s 661ms/step - loss: 0.3259
Epoch 3/10
36/36 [==============================] - 24s 658ms/step - loss: 0.3195
Epoch 4/10
36/36 [==============================] - 24s 665ms/step - loss: 0.2775
Epoch 5/10
36/36 [==============================] - 24s 659ms/step - loss: 0.2396
Epoch 6/10
 2/36 [>.............................] - ETA: 24s - loss: 0.2177

# Save scores

In [ ]:
path_scores = './Scores'
model_name = 'cnn'

In [ ]:
# stratified 5-fold
with open(f'{path_scores}/strat_5_fold//{model_name}.txt', 'w') as dat:
    dat.write(str(scores_strat_group_k_fold))

In [ ]:
# stratified group 5-fold
with open(f'{path_scores}/strat_group_5_fold//{model_name}.txt', 'w') as dat:
    dat.write(str(scores_strat_k_fold))

In [ ]:
# LOSO
with open(f'{path_scores}/strat_5_loso_fold//{model_name}.txt', 'w') as dat:
    dat.write(str(scores_loso))

# Under development

train/test split

In [14]:
variable = 0

model = CNN(path, variable=0, epochs=10, learning_rate=1e-3, batch_size=16)

scores = stratified_train_test(path, model, test_size=0.2, images=True, verbose=True, variable=0)

Starting stratified train/test for physical fatigue


C:\Users\jjung\anaconda3\envs\tf-sklearn-keras\lib\site-packages\keras\engine\training.py:1972: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/10
144/144 [==============================] - 71s 315ms/step - loss: 0.3806
Epoch 2/10
144/144 [==============================] - 44s 304ms/step - loss: 0.3052
Epoch 3/10
144/144 [==============================] - 44s 304ms/step - loss: 0.3001
Epoch 4/10
144/144 [==============================] - 44s 305ms/step - loss: 0.2252
Epoch 5/10
144/144 [==============================] - 44s 305ms/step - loss: 0.2365
Epoch 6/10
144/144 [==============================] - 44s 305ms/step - loss: 0.2089
Epoch 7/10
144/144 [==============================] - 44s 304ms/step - loss: 0.1853
Epoch 8/10
144/144 [==============================] - 44s 304ms/step - loss: 0.2005
Epoch 9/10
144/144 [==============================] - 44s 305ms/step - loss: 0.1883
Epoch 10/10
144/144 [==============================] - 44s 305ms/step - loss: 0.1625
Performance model:
 accuracy: 0.808 +- 0.808 

 balanced_accuracy: 0.801 +- 0.801 

 f1: 0.816 +- 0.816 

 recall: 0.808 +- 0.808 

 precision: 0.836 +- 0.836 

Daily majority vote

In [15]:
# test set
test_indices = model.test_indices

# day information for each segment
'''metadata_test = np.array(metadata)[test_indices]
days_test = [(meta['date'], meta['subjectID']) for meta in metadata_test] # note that same date can be used by different subjects'''

# true labels
y = np.empty(len(test_indices), dtype=int)
for i, index in enumerate(test_indices):
    y_i = np.load(path + f'/labels{index}.npy', allow_pickle=True)[0]
    y[i] = y_i

# model predictions
y_pred_segments = model.predict(test_indices)
y_pred_segments = y_pred_segments.numpy().astype(int)

In [16]:
daily_majority_vote(y_pred_segments, test_indices, metadata)

{('09.02.18', 24): 1,
 ('09.05.19', 9): 0,
 ('27.07.18', 24): 1,
 ('27.01.18', 26): 1,
 ('07.05.19', 9): 1,
 ('12.01.18', 23): 0,
 ('07.06.19', 15): 1,
 ('28.12.17', 23): 1,
 ('01.01.18', 23): 0,
 ('06.04.19', 5): 1,
 ('28.02.18', 26): 1,
 ('11.02.18', 23): 1,
 ('16.02.18', 23): 0,
 ('10.05.18', 24): 1,
 ('23.01.18', 23): 1,
 ('14.02.18', 23): 1,
 ('31.03.18', 24): 0,
 ('26.02.18', 23): 1,
 ('10.05.19', 9): 0,
 ('16.03.19', 1): 0,
 ('20.05.19', 16): 1,
 ('29.01.18', 26): 1,
 ('06.05.19', 7): 1,
 ('21.05.19', 16): 1,
 ('01.09.18', 27): 1,
 ('19.05.18', 24): 1,
 ('20.02.18', 23): 0,
 ('12.06.19', 8): 1,
 ('15.05.19', 11): 0,
 ('13.03.18', 24): 1,
 ('11.05.19', 9): 1,
 ('10.03.18', 24): 0,
 ('01.03.18', 24): 1,
 ('16.07.18', 24): 1,
 ('09.05.19', 11): 0,
 ('13.05.19', 11): 0,
 ('31.12.17', 23): 1,
 ('10.07.19', 21): 1,
 ('04.03.18', 24): 1,
 ('26.03.19', 4): 0,
 ('02.01.18', 24): 1,
 ('07.04.19', 5): 1,
 ('17.06.19', 14): 1,
 ('19.06.19', 13): 0,
 ('11.02.18', 25): 1,
 ('22.01.18', 23): 1

In [17]:
agreements(y_pred_segments, test_indices, metadata)

0.8035714285714286

In [19]:
variable = 0

model = CNN(path, variable=0, epochs=10, learning_rate=1e-3, batch_size=16)

In [20]:
N = sum([1 for p in os.listdir(path) if (p[:14] == 'feature_vector' and p[:19] != 'feature_vector_stat')])

# load labels (we need them for stratification)
y = np.empty(N, dtype=int)
for i in range(N):
    y[i] = np.load(path + f'/labels{i}.npy', allow_pickle=True)[variable]  # TODO: multiclass

# CV
cv = StratifiedGroupKFold(n_splits=5, shuffle=True, random_state=SEED)
scores_cv = []
data_indices = np.arange(N)

print(f'Starting stratified group {5}-fold for {["physical fatigue", "mental fatigue"][variable]}')
with tqdm(total=5) as pbar:
    for i, (train_indices, test_indices) in enumerate(cv.split(X=data_indices, y=y, groups=subjects)):
        # test labels
        y_test = y[test_indices]

        # training
        model.reset()
        model.fit(train_indices)

        # predict
        y_pred = model.predict(test_indices)

        # evaluate
        scores = evaluator(y_pred, y_test, verbose=False)
        scores_cv.append(scores)

        # agreements
        print(agreements(y_pred, model.test_indices, metadata))

        # for progress bar
        pbar.update(1)
        pbar.set_description(f' Fold {i+1} F1: {scores["f1"]}')

# print (if verbose==True)
print('Performance model:')
metrics = scores_cv[0].keys()
for metric in metrics:
    mean = np.mean([scores_cv_i[metric] for scores_cv_i in scores_cv])
    std = np.std([scores_cv_i[metric] for scores_cv_i in scores_cv])
    print(f' {metric}: {round(mean, 3)} +- {round(std, 3)} \n')

Starting stratified group 5-fold for physical fatigue


  0%|          | 0/5 [00:00<?, ?it/s]

C:\Users\jjung\anaconda3\envs\tf-sklearn-keras\lib\site-packages\keras\engine\training.py:1972: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/10
138/138 [==============================] - 68s 314ms/step - loss: 0.3386
Epoch 2/10
138/138 [==============================] - 41s 298ms/step - loss: 0.2392
Epoch 3/10
138/138 [==============================] - 41s 297ms/step - loss: 0.2046
Epoch 4/10
138/138 [==============================] - 41s 298ms/step - loss: 0.1841
Epoch 5/10
138/138 [==============================] - 41s 299ms/step - loss: 0.1494
Epoch 6/10
138/138 [==============================] - 41s 298ms/step - loss: 0.1477
Epoch 7/10
138/138 [==============================] - 41s 298ms/step - loss: 0.1307
Epoch 8/10
138/138 [==============================] - 41s 297ms/step - loss: 0.1179
Epoch 9/10
138/138 [==============================] - 41s 297ms/step - loss: 0.1070
Epoch 10/10
138/138 [==============================] - 41s 298ms/step - loss: 0.1097


 Fold 1 F1: 0.5681694894300688:  20%|██        | 1/5 [09:09<36:39, 549.81s/it]

...

0.3811881188118812


C:\Users\jjung\anaconda3\envs\tf-sklearn-keras\lib\site-packages\keras\engine\training.py:1972: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/10
149/149 [==============================] - 70s 301ms/step - loss: 0.2720
Epoch 2/10
149/149 [==============================] - 44s 296ms/step - loss: 0.2188
Epoch 3/10
149/149 [==============================] - 44s 295ms/step - loss: 0.1866
Epoch 4/10
149/149 [==============================] - 44s 297ms/step - loss: 0.1476
Epoch 5/10
149/149 [==============================] - 45s 298ms/step - loss: 0.1693
Epoch 6/10
149/149 [==============================] - 44s 297ms/step - loss: 0.1234
Epoch 7/10
149/149 [==============================] - 44s 296ms/step - loss: 0.1132
Epoch 8/10
149/149 [==============================] - 44s 294ms/step - loss: 0.1109
Epoch 9/10
149/149 [==============================] - 45s 299ms/step - loss: 0.0945
Epoch 10/10
149/149 [==============================] - 44s 296ms/step - loss: 0.0937


 Fold 5 F1: 0.5310234571376662: 100%|██████████| 5/5 [48:52<00:00, 586.58s/it]

0.3835616438356164
Performance model:
 accuracy: 0.59 +- 0.067 

 balanced_accuracy: 0.575 +- 0.055 

 f1: 0.605 +- 0.06 

 recall: 0.59 +- 0.067 

 precision: 0.651 +- 0.118 



In [14]:
variable = 0
model = CNN(path, variable=0, epochs=10, learning_rate=1e-3, batch_size=16)

N = sum([1 for p in os.listdir(path) if (p[:14] == 'feature_vector' and p[:19] != 'feature_vector_stat')])

# load labels (we need them for stratification)
y = np.empty(N, dtype=int)
for i in range(N):
    y[i] = np.load(path + f'/labels{i}.npy', allow_pickle=True)[variable]  # TODO: multiclass

# CV
cv = StratifiedGroupKFold(n_splits=5, shuffle=True, random_state=SEED)
scores_cv = []
data_indices = np.arange(N)

print(f'Starting stratified group {5}-fold for {["physical fatigue", "mental fatigue"][variable]}')
with tqdm(total=5) as pbar:
    for i, (train_indices, test_indices) in enumerate(cv.split(X=data_indices, y=y, groups=subjects)):
        # test labels
        y_test = y[test_indices]

        # training
        model.reset()
        model.fit(train_indices)

        # predict
        y_pred = model.predict(test_indices)

        # daily majorities
        y_pred_daily = np.array(list(daily_majority_vote(y_pred, test_indices, metadata).values()))
        y_true_daily = np.array(list(daily_majority_vote(y_test, test_indices, metadata).values()))

        # evaluate
        scores = evaluator(y_pred_daily, y_true_daily, verbose=False)
        scores_cv.append(scores)

        # agreements
        print('agreements:', agreements(y_pred, model.test_indices, metadata))

        # for progress bar
        pbar.update(1)
        pbar.set_description(f' Fold {i+1} F1: {scores["f1"]}')

# print (if verbose==True)
print('Performance model:')
metrics = scores_cv[0].keys()
for metric in metrics:
    mean = np.mean([scores_cv_i[metric] for scores_cv_i in scores_cv])
    std = np.std([scores_cv_i[metric] for scores_cv_i in scores_cv])
    print(f' {metric}: {round(mean, 3)} +- {round(std, 3)} \n')

Starting stratified group 5-fold for physical fatigue


  0%|          | 0/5 [00:00<?, ?it/s]

C:\Users\jjung\anaconda3\envs\tf-sklearn-keras\lib\site-packages\keras\engine\training.py:1972: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/10
138/138 [==============================] - 68s 332ms/step - loss: 0.3020
Epoch 2/10
138/138 [==============================] - 41s 293ms/step - loss: 0.2282
Epoch 3/10
138/138 [==============================] - 41s 293ms/step - loss: 0.2173
Epoch 4/10
138/138 [==============================] - 41s 292ms/step - loss: 0.1672
Epoch 5/10
138/138 [==============================] - 41s 293ms/step - loss: 0.1465
Epoch 6/10
138/138 [==============================] - 40s 292ms/step - loss: 0.1315
Epoch 7/10
138/138 [==============================] - 40s 292ms/step - loss: 0.1265
Epoch 8/10
138/138 [==============================] - 40s 292ms/step - loss: 0.1192
Epoch 9/10
138/138 [==============================] - 40s 292ms/step - loss: 0.1153
Epoch 10/10
138/138 [==============================] - 40s 292ms/step - loss: 0.1095


 Fold 1 F1: 0.5801763453937365:  20%|██        | 1/5 [09:04<36:19, 544.78s/it]

...

agreements: 0.4801980198019802
agreements (aggregate): 0.7970297029702971


C:\Users\jjung\anaconda3\envs\tf-sklearn-keras\lib\site-packages\keras\engine\training.py:1972: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/10
149/149 [==============================] - 72s 332ms/step - loss: 0.3174
Epoch 2/10
149/149 [==============================] - 50s 334ms/step - loss: 0.2267
Epoch 3/10
149/149 [==============================] - 49s 327ms/step - loss: 0.1960
Epoch 4/10
149/149 [==============================] - 51s 344ms/step - loss: 0.1866
Epoch 5/10
149/149 [==============================] - 51s 344ms/step - loss: 0.1458
Epoch 6/10
149/149 [==============================] - 49s 325ms/step - loss: 0.1324
Epoch 7/10
149/149 [==============================] - 49s 328ms/step - loss: 0.1209
Epoch 8/10
149/149 [==============================] - 49s 329ms/step - loss: 0.1121
Epoch 9/10
149/149 [==============================] - 49s 327ms/step - loss: 0.1007
Epoch 10/10
149/149 [==============================] - 49s 329ms/step - loss: 0.0972


 Fold 5 F1: 0.3568734427779364: 100%|██████████| 5/5 [50:41<00:00, 608.36s/it]

agreements: 0.9178082191780822
agreements (aggregate): 0.958904109589041
Performance model:
 accuracy: 0.582 +- 0.112 

 balanced_accuracy: 0.553 +- 0.108 

 f1: 0.551 +- 0.16 

 recall: 0.582 +- 0.112 

 precision: 0.608 +- 0.181 



# TODO: check why so few days (even fewer than statistical features)